In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.patches
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox,Label
import plotly as ply
from dance_sim_tools import trace_objects,utility
from dance_sim_tools.utility import histogram_draw_to_parameters as htp
from dance_sim_tools.utility import draw_reversal_distances_tm
from dance_sim_tools.ipywidget_helpers import slider
import pandas as pd
import dance_sim_tools.trace_objects as trace_objects
import warnings
warnings.filterwarnings('ignore')
import scipy.stats
import matplotlib.gridspec

In [2]:
#Pull up all of the r_1 and r_0 for each three conditions 

#we collect all the ks and the cs for each of the 3 conditions

def zero_insert(a):
    return np.hstack((np.array([0]),a))

def get_c_hist(m,logN,n_bins):

    n_bins = int(n_bins)
    m = int(m)     #subsample size out of all the flies in the condition
    N = int(np.exp(logN))
    cs = np.full((3,N),np.nan)
 
    for condition in [1,2,3]:

        r_0_filename = 'r_0andr_1/r_0_'+str(condition)+'.txt'    
        empirical_r0s = pd.read_csv(r_0_filename).values.T[0]#, header=None, usecols=[2])

        r_1_filename = 'r_0andr_1/r_1_'+str(condition)+'.txt'    
        empirical_r1s = pd.read_csv(r_1_filename).values.T[0]#, header=None, usecols=[2])

        #For N iterations, draw m pairs of r_1,r_0, and mean(r_1-r_0) using only the subsample
        for i in range(N):
            #randomly select m r_1,r_0 pairs from all r_1s,r_0s for the condition
            #draw m of r_1
            r_1_draw_idx = np.arange(len(empirical_r1s))
            np.random.shuffle(r_1_draw_idx)
            r_1_draw = empirical_r1s[r_1_draw_idx[:m]]
            #Find the r_0 with the same index (=fly index) as those drawn above
            r_0_draw = empirical_r0s[r_1_draw_idx[:m]]

            #Just compute the mean r1-r0 for the subsample
            cs[condition-1,i] = np.mean(r_1_draw-r_0_draw)
             
#             fit_stds[condition-1,i] = np.std(np.degrees(r_1_draw-r_0_draw))


    #Compute and store the histogram of cs for each condition
    cmin,cmax = 0,180
    cbins = np.linspace(cmin,cmax,n_bins)
    cs = np.degrees(cs)    

    
    c_probs = np.zeros((3,len(cbins)-1))
    for i in range(3):
        n,_=np.histogram(cs[i,:],bins=cbins)
        c_probs[i,:]=n/np.sum(n)
    
    return cbins,c_probs
  

def get_k_hist(logN,n_bins):
    n_bins = int(n_bins)
    N = int(np.exp(logN))
    ks = np.full((3,N),np.nan)
 
    for i,condition in enumerate(['1F_1','2F_60_2','2F_90_3']):

        k_filename = 'k_inter_rev/k_inter_'+str(condition)+'.txt'    
        empirical_ks = np.loadtxt(k_filename)#, header=None, usecols=[0,1]).values.T[0]
        #the imported array empirical_ks has shape flies x 30, ( = up to 30 reversal k values)
        fly_count = np.shape(empirical_ks)[0]
        nan_empirical_ks = np.copy(empirical_ks)
        nan_empirical_ks[nan_empirical_ks<0.01] = np.nan
        fly_mean_ks = np.nanmean(nan_empirical_ks,axis=1)
        ks[i,:fly_count] = fly_mean_ks
        
     #Compute and store the histogram of ks for each condition
    kmin,kmax = 0,3.
    kbins = np.linspace(kmin,kmax,n_bins)
    
    k_probs = np.zeros((3,len(kbins)-1))
    for i in range(3):
        n,_=np.histogram(ks[i,:],bins=kbins)
        k_probs[i,:]=n/np.sum(n)
    
    return kbins,k_probs
  
def get_adjusted_emp_r1_hist(n_bins):
    emp_r1s = np.full((3,30),np.nan)
    for i,condition in enumerate(['1','2','3']):

        emp_r1_filename = 'r_0andr_1adjusted/r_1_'+str(condition)+'.txt'    
        empirical_r1s = pd.read_csv(emp_r1_filename).values.T[0]#, header=None, usecols=[2])

        #the imported array empirical_ks has shape flies x 30, ( = up to 30 reversal k values)
        fly_count = np.shape(empirical_r1s)[0]
        emp_r1s[i,:fly_count] = empirical_r1s
        
     #Compute and store the histogram of ks for each condition
    emp_r1_min,emp_r1_max = 40,200.
    emp_r1_bins = np.linspace(emp_r1_min,emp_r1_max,n_bins)
    emp_r1_probs = np.zeros((3,len(emp_r1_bins)-1))
    
    for i in range(3):
        n,_=np.histogram(emp_r1s[i,:],bins=emp_r1_bins)
        emp_r1_probs[i,:]=n/np.sum(n)    
    return emp_r1_bins,emp_r1_probs
    
    
def reversal_count_dist(n_bins):
    reversal_count_bins = np.arange(0,31,1)
    reversal_count_probs = np.zeros((3,30))

    for i,condition in enumerate(['1F_1','2F_60_2','2F_90_3']):

        k_filename = 'k_inter_rev/k_inter_'+str(condition)+'.txt'    
        empirical_ks = np.loadtxt(k_filename)#, header=None, usecols=[0,1]).values.T[0]
        #the imported array empirical_ks has shape flies x 30, ( = up to 30 reversal k values)
        fly_count = np.shape(empirical_ks)[0]
        nan_empirical_ks = np.copy(empirical_ks)
        nan_empirical_ks[nan_empirical_ks<0.01] = np.nan
        reversal_count_vals = np.sum(~np.isnan(nan_empirical_ks),axis=1)
        reversal_count_ns,_ = np.histogram(reversal_count_vals,bins=reversal_count_bins)
        reversal_count_probs[i,:30] = reversal_count_ns/np.sum(reversal_count_ns)
        
        
    #reversal_counts is the same for every condition
    #reversal_count_probs is different for each condition    
    
    return reversal_count_bins,reversal_count_probs

## Trace Generation w/ Mechanistic Two-Part Reversal Distance Model

Below: we generate a single straight line trace according to the 
inter-reversal distance relationship:

$r_{1}$ drawn according to the process described in neural_model_0.ipynb

$r_{i} = kr_{i-1}, i>1$, except that it is implemented with a threshold mechanism as well.


In [3]:
def g(r_0,c_0,c_1,c_2,kappa,logM,
      two_step=True,dt=0.25,condition=0,
      n_bins=40,velocity=10,t_stop=4.,var_type=True,constant_var=0):
        
    #Make the trace
    num_segments = 70
    
#     r_0 = np.radians(r_0)
#     velocity = np.radians(velocity)



    if condition==1:
        food1_loc = 0
        c = c_0
    if condition==2:
        food1_loc = -30
        c = c_1
    if condition==3:
        food1_loc = -45
        c = c_2

    
    food1_loc = np.radians(food1_loc)
    
    
    
    n_bins = int(n_bins)
    M = int(np.exp(logM)) #M: number of traces 

    if two_step:
        r_0_est = draw_reversal_distances_tm(M,r_0,velocity,dt,kappa,vdv=var_type,constant_var=constant_var)  #Draw the r_0 values
        c_est = draw_reversal_distances_tm(M,c,velocity,dt,kappa,vdv=var_type,constant_var=constant_var)  #Draw the c values
        r_1 = r_0_est+c_est
    else:
        r_1s = draw_reversal_distances_tm(M,r_0+c,velocity,dt,kappa,vdv=var_type,constant_var=constant_var)  #Draw the r_1 values
    #-------------------------------------
        
        
        
        
    reversal_bin_count_array = np.zeros((M,n_bins))
    reversal_count_bins,reversal_count_probs = reversal_count_dist(n_bins)
    r_is = np.full((M,30),np.nan)
    
    
    for i in range(M):
        n_ki_draw = int(np.random.choice(reversal_count_bins[:-1],p=reversal_count_probs[condition-1,:]))
#         n_ki_draw = 15
        irds = np.zeros(n_ki_draw)
        irds[0] = r_1[i]
        irds[1:] = draw_reversal_distances_tm(n_ki_draw-1,r_0+c,velocity,dt,kappa)
        r_is[i,0:(n_ki_draw-1)] = irds[1:]
        trace = trace_objects.DistanceListTrace(
                food1_loc,np.radians(irds),np.radians(r_0),t_stop=t_stop,
                    velocity=np.radians(velocity),num_transit_ticks=n_bins)
        reversals_inds,reversals_bin_counts = trace.find_reversal_inds()
#         print(np.shape(reversal_bin_count_array))
#         print(np.shape(reversals_bin_counts))
#         print(np.degrees(trace.location_bins_rads))
    
        reversal_bin_count_array[i,:] = reversals_bin_counts
        
    transit_tick_interval = 2*np.pi/n_bins
    
    x = trace.t
    y = np.degrees(trace.path)
    
    fig = plt.figure(1,figsize=(12,20))#,dpi=200)
        
    gs = matplotlib.gridspec.GridSpec(6,4,hspace=0.5,wspace=0.5)

    ax1 = fig.add_subplot(gs[0:2,0:2])
    plt.plot(x,y,'o',color='grey',markersize=1,zorder=1)
    plt.rc('text', usetex=True)
    plt.xlabel('Time (min)')
    theta_1,theta_2 = np.degrees(food1_loc),-1*np.degrees(food1_loc)
    t_1,t_2 = 0,max(trace.t)
    plt.plot([t_1,t_2],[theta_2,theta_2],color='green')
    plt.plot([t_1,t_2],[theta_1,theta_1],color='green')

    yticks = np.arange(-180,210,30)
    plt.yticks(yticks)

    plt.ylim([-180.,180.]) 
    plt.scatter(x[reversals_inds],y[reversals_inds],color='b',zorder=2)
    
    #Plot the projections of the reversals onto the right y axis
    left_dots = np.vstack((x[reversals_inds],y[reversals_inds]))
    right_dots = np.vstack((t_stop*np.ones_like(x[reversals_inds]),y[reversals_inds]))
    #shape outer_dots is (2,num_spots)
    whole = np.stack((left_dots,right_dots))
    #shape whole is (2,2,num_spots)
    plt.plot(whole[:,0,:],whole[:,1,:],color='pink',zorder=0);
    #The main takeaway is to make one big array whose dimensions are
    #2 (inner vs outer ) by
    #2 (x and y) by
    #num angles
    
    ax2 = fig.add_subplot(gs[0:2,2:4])# plt.subplot(222)
    y = np.degrees(trace.location_bins_rads)
    bar_values = np.sum(reversal_bin_count_array,axis=0)
    special_ind_bool = (bar_values==np.max(bar_values)) 
    bar_values[special_ind_bool] = 0.
    bar_values[special_ind_bool] = np.max(bar_values)
    bar_values = bar_values/(np.sum(bar_values))
    bin_width=y[1]-y[0]
    ax2.barh(y[:-1],bar_values,height=bin_width,align='edge')
    plt.title('Reversal Counts')
    plt.ylim([-180.,180.])
    plt.xlabel('Fraction of Reversals')

    yticks = np.arange(-180,210,30)
    plt.yticks(yticks)#,y_ticklabels,fontsize=15,verticalalignment='bottom')
    
   
    
    if condition==1: #1F
        f1_dot = matplotlib.patches.Rectangle(
                (-0.05,0.5-(7.5/360)), .05,(15./360),
                label='food',fc="g",transform=ax2.transAxes,clip_on=False)
        ax2.add_artist(f1_dot)
        plt.axhspan(-7.5,7.5, facecolor='g', alpha=0.3,label='food') 
    if condition>1: #2F
#         theta_1,theta_2 = 0.5+food1_loc/180,0.5-1*food1_loc/180
        theta_1_loc,theta_2_loc = 0.5+np.degrees(food1_loc)/180*0.5,0.5+-1*(np.degrees(food1_loc)/180)*0.5
        f1_dot = matplotlib.patches.Rectangle((-0.05,theta_1_loc-(7.5/360)), .05,(15./360), fc="g",transform=ax2.transAxes,clip_on=False)
        f2_dot = matplotlib.patches.Rectangle((-0.05,theta_2_loc-(7.5/360)), .05,(15./360), fc="g",transform=ax2.transAxes,clip_on=False) 
   
        ax2.add_artist(f1_dot)
        ax2.add_artist(f2_dot)
        theta_1,theta_2 = np.degrees(food1_loc),-1*np.degrees(food1_loc)
        plt.axhspan(theta_1-(7.5), theta_1+(7.5), facecolor='g', alpha=0.3,label='food') 
        plt.axhspan(theta_2-(7.5), theta_2+(7.5), facecolor='g', alpha=0.3) 


        
    plt.legend(bbox_to_anchor=(1.3,.5))
    
    #New middle plot for the vertical version of the reversal histogram
    axa = fig.add_subplot(gs[2:4,2:4])
    axa.plot(y[:-1]+bin_width/2,bar_values,'-o')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        labelleft=False) # labels along the bottom edge are off    
    plt.title('Reversal Counts')
    plt.xlim([-180.,180.])
    plt.ylabel('Fraction of Reversals')
    xticks = np.arange(-180,210,30)
    plt.xticks(xticks)#,y_ticklabels,fontsize=15,verticalalignment='bottom')
    
    
    if condition==1: #1F
        f1_dot = matplotlib.patches.Rectangle(
                (0.5-(7.5/360),-0.05), .05,(15./360),
                label='food',fc="g",transform=axa.transAxes,clip_on=False)
        axa.add_artist(f1_dot)
        plt.axvspan(-7.5,7.5, facecolor='g', alpha=0.3,label='food') 
    if condition>1: #2F
        theta_1_loc,theta_2_loc = 0.5+np.degrees(food1_loc)/180*0.5,0.5+-1*(np.degrees(food1_loc)/180)*0.5
        f1_dot = matplotlib.patches.Rectangle((theta_1_loc-(7.5/360),-0.05), .05,(15./360), fc="g",transform=axa.transAxes,clip_on=False)
        f2_dot = matplotlib.patches.Rectangle((theta_2_loc-(7.5/360),-0.05), .05,(15./360), fc="g",transform=axa.transAxes,clip_on=False) 
        axa.add_artist(f1_dot)
        axa.add_artist(f2_dot)
        theta_1,theta_2 = np.degrees(food1_loc),-1*np.degrees(food1_loc) 
        plt.axvspan(theta_1-(7.5), theta_1+(7.5), facecolor='g', alpha=0.3,label='food') 
        plt.axvspan(theta_2-(7.5), theta_2+(7.5), facecolor='g', alpha=0.3) 

    
    
        
    ax3 = fig.add_subplot(gs[4:6,0:2],polar=True)#plt.subplot(223,polar=True)
    plt.plot(trace.path,trace.t)
    ax3.set_theta_zero_location('S', offset=0)
    n_ticks = 12
    tick_degree_values = np.linspace(0,360-(360/n_ticks),n_ticks)
    ax3.set_xticks(np.radians(tick_degree_values))
    pt1 =  np.linspace(0,180-(360/n_ticks),n_ticks/2).astype(int)
    pt2 =  np.linspace(-180,0-(360/n_ticks),n_ticks/2).astype(int)
    ax3.set_xticklabels(np.hstack((pt1,pt2)))
    theta_1,theta_2 = food1_loc,-1*food1_loc
    r_1,r_2 = 0,max(trace.t)
    plt.plot([theta_2,theta_2],[r_1,r_2],color='green')
    plt.plot([theta_1,theta_1],[r_1,r_2],color='green')

    
    
    ax3.set_yticklabels([])
    
    alpha=0.2
    colors = ['blue','orange','red']

    
    ###----------r_1 Distribution----------
    
    r_1_bins = np.linspace(40,200,50)
    ax4 = fig.add_subplot(gs[4:5,2:4])
    plt.xlabel('Distance (Degrees)')
    plt.title('r1 Distributions')
    plt.xticks(np.linspace(0,180,7))
 
    bin_width = r_1_bins[1]-r_1_bins[0]
    n,_ = np.histogram(r_1,bins=r_1_bins)
    plt.bar(r_1_bins[:-1],n/np.sum(n),align='edge',width=bin_width)
    
    emp_r1_vals,emp_r1_probs = get_adjusted_emp_r1_hist(n_bins)
    
    plt.step(emp_r1_vals,zero_insert(emp_r1_probs[0,:]),color=colors[0],label='1F',alpha=(0.2+(0.6)*int(condition==1)))
    plt.step(emp_r1_vals,zero_insert(emp_r1_probs[1,:]),color=colors[1],label='2F60',alpha=(0.2+(0.8)*int(condition==2)))
    plt.step(emp_r1_vals,zero_insert(emp_r1_probs[2,:]),color=colors[2],label='2F90',alpha=(0.2+(0.8)*int(condition==3)))
    plt.legend(title='Empirical r1 Distribution',bbox_to_anchor=(1.1,0.5))
    
    
    n,_ = np.histogram(r_is[~np.isnan(r_is)],bins=r_1_bins)
    plt.bar(r_1_bins[:-1],n/np.sum(n),align='edge',
            width=bin_width,color=colors[0],label='model')
   
    
    ###----------r_is Distribution----------

    ax4a = fig.add_subplot(gs[5:6,2:4])# plt.subplot(224)
    plt.title('ris Distributions')
    plt.xlabel('Distance (Degrees)')
    plt.xticks(np.linspace(0,180,7))
    n,_ = np.histogram(r_is[~np.isnan(r_is)],bins=r_1_bins)
    plt.bar(r_1_bins[:-1],n/np.sum(n),align='edge',width=bin_width)
  
          
     
      
condition_toggle = widgets.RadioButtons(options=[('1F',1), ('2F 60',2),
                                          ('2F 90',3)],disabled=False,description='condition')     
two_step_toggle  = widgets.RadioButtons(options=[('True',1),('False',0)],disabled=False,description='two_step')
var_type_toggle  = widgets.RadioButtons(options=[('Value-Dependent',1),('Independent',0)],disabled=False,description='var_type')


#start,stop,step,init

sr_0=slider('r_0',20.,80.,5.,40.)
sc_0= slider('c_0',10,180,10,40)
sc_1= slider('c_1',10,180,10,80)
sc_2= slider('c_2',10,180,10,110)
skappa=slider('kappa',0.00,.2,0.01,0.02,readout_format='.2f')
slogM=slider('logM',3,9,1,5)
sdt=slider('dt',0.01,0.5,0.01,0.25,readout_format='.2f')
sn_bins =  slider('n_bins',5,100,1,30)
svelocity=slider('velocity',1,15,1,7)
st_stop = slider('t_stop',0.,5.,.1,3.)
sconstant_var = slider('constant_var',0.,10.,.1,1.,readout_format='.2f')

sliders = [sr_0,sc_0,sc_1,sc_2,skappa,slogM,sdt,sn_bins,svelocity,st_stop,
           condition_toggle,two_step_toggle,var_type_toggle,sconstant_var]
           
items = [Box([slider]) for slider in sliders]
  

ui = Box(items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='30%'
))

slider_names = [slider.description for slider in sliders]
param_dict =  dict(zip(slider_names,sliders))


out = widgets.interactive_output(g, param_dict)



display(ui,out)



Box(children=(Box(children=(FloatSlider(value=40.0, continuous_update=False, description='r_0', max=80.0, min=…

Output()

Note: in the top right figure above, we chop off the bin count on the bin containing $f_1-r_0$ so that it is equal to the height of the second highest bin. (because r_0 is always fixed, the first reversal will always fall in the same bin).

Now, do a modified version of the above in which we draw N times (N traces), where each trace is a sample from the distribution of c values for the condition. 